# Twisted Bilayer Boron Nitride (TBBN) Structure Creation

## 0. Introduction

This tutorial demonstrates the process of creating a twisted bilayer boron nitride structure as described in:

> **Lede Xian, Dante M. Kennes, Nicolas Tancogne-Dejean, Massimo Altarelli, and Angel Rubio**, "Multiflat Bands and Strong Correlations in Twisted Bilayer Boron Nitride: Doping-Induced Correlated Insulator and Superconductor"

We will reproduce the structure shown in Figure 1(a,b) of the paper, which shows a twisted bilayer hBN with specific stacking regions.

Key features to reproduce:
1. Hexagonal boron nitride bilayer structure
2. Twisted configuration with specified angle
3. Proper stacking of A/B regions

The image shows two possible configurations in twisted bilayer hBN with B/B regions (highlighted with solid blue circle) and B/N regions (highlighted with dash red circle).


## 1. Prepare the Environment
### 1.1. Set up structure parameters 


In [ ]:
# Material and structure parameters
TWIST_ANGLE = 2.64  # Twist angle in degrees (from paper)

# Nanoribbons parameters

RIBBON_WIDTH = 100  # Width of the nanoribbon in Angstroms
RIBBON_LENGTH = 100  # Length of the nanoribbon in Angstrom

# Interface parameters
INTERFACE_DISTANCE = 3.0  # Distance between layers in Angstroms
INTERFACE_VACUUM = 20.0  # Vacuum layer thickness in Angstroms


# Visualization settings
SHOW_INTERMEDIATE_STEPS = True
VISUALIZE_REPETITIONS = [1, 1, 1]


### 1.2. Install Packages


In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip
  
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("", "../../config.yml")


### 1.3. Get input material
We'll use hexagonal boron nitride (h-BN) as our starting material.


In [ ]:
from mat3ra.made.material import Material
from mat3ra.standata.materials import Materials
from utils.visualize import visualize_materials

material = Material(Materials.get_by_name_first_match("BN"))

print("Initial material properties:")
print(f"Formula: {material.formula}")
print(f"Number of atoms: {len(material.basis.elements.ids)}")
print(f"Lattice vectors:\n{material.basis.cell.vectors_as_array}")

if SHOW_INTERMEDIATE_STEPS:
    visualize_materials(material, repetitions=VISUALIZE_REPETITIONS)


## 2. Create Twisted Bilayer Structure
### 2.1. Set up interface configuration and builder


In [ ]:
from mat3ra.made.tools.build.interface.builders import NanoRibbonTwistedInterfaceBuilder
from mat3ra.made.tools.build.interface.configuration import NanoRibbonTwistedInterfaceConfiguration

config = NanoRibbonTwistedInterfaceConfiguration(
    film=material,
    substrate=material, 
    twist_angle=TWIST_ANGLE,
    distance_z=INTERFACE_DISTANCE,
    ribbon_width=RIBBON_WIDTH,
    ribbon_length=RIBBON_LENGTH,
)


builder = NanoRibbonTwistedInterfaceBuilder()


### 2.2. Generate and analyze interfaces


In [ ]:
# Generate interfaces
interfaces = builder.get_materials(config)

print(f"\nFound {len(interfaces)} possible interface(s)")
for i, interface in enumerate(interfaces):
    actual_angle = interface.metadata.get("actual_twist_angle", "unknown")
    print(f"\nInterface {i+1}:")
    print(f"Actual twist angle: {actual_angle}°")
    print(f"Number of atoms: {len(interface.basis.elements.ids)}")
    print(f"Cell vectors:\n{interface.basis.cell.vectors_as_array}")


### 2.3. Select and visualize the best interface


In [ ]:
if len(interfaces) == 0:
    print("No interfaces found! Try adjusting the search parameters.")
else:
    # Select interface closest to target angle
    angles = [interface.metadata.get("actual_twist_angle", 0) for interface in interfaces]
    angle_diffs = [abs(angle - TWIST_ANGLE) for angle in angles]
    best_index = angle_diffs.index(min(angle_diffs))
    
    selected_interface = interfaces[best_index]
    actual_angle = selected_interface.metadata.get("actual_twist_angle", "unknown")
    
    print(f"Selected interface {best_index + 1}:")
    print(f"Target angle: {TWIST_ANGLE}°")
    print(f"Actual angle: {actual_angle}°")
    print(f"Number of atoms: {len(selected_interface.basis.elements.ids)}")
    
    visualize_materials(selected_interface, repetitions=VISUALIZE_REPETITIONS)
    visualize_materials(selected_interface, repetitions=VISUALIZE_REPETITIONS, rotation="-90x")


## 5. Pass data to the outside runtime


In [ ]:
from utils.jupyterlite import write_materials_to_folder

write_materials_to_folder(selected_interface, "Twisted Bilayer Boron Nitride.json")